In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
import os.path
import info
from utils import *
from backtest_algos import *

warnings.filterwarnings("ignore")

path_pc = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/'
today = datetime.today()
print("Today's date:", today.strftime("%Y-%m-%d"))

Today's date: 2020-04-20


In [2]:
outcomes_new = load_latest(today, 'outcomes_new_features_', path_pc)

Trying  outcomes_new_features_2020-04-20.csv
Trying  outcomes_new_features_2020-04-19.csv
Trying  outcomes_new_features_2020-04-18.csv
Trying  outcomes_new_features_2020-04-17.csv
Loading file:  outcomes_new_features_2020-04-17.csv


In [3]:
outcomes_new.tail()

(close/ema)-1  (close/sma)-1  1-lower/close  50d_high  \
date       symbol                                                           
2020-04-17 2382.HK      -0.027032      -0.041699       0.168619    134.00   
           2388.HK      -0.002752      -0.019886       0.173541     27.95   
           2628.HK      -0.036590      -0.048205       0.203558     19.78   
           3328.HK      -0.009745      -0.013387       0.086737      5.22   
           3988.HK      -0.013242      -0.018316       0.063913      3.23   

                    50d_high_volume     atr10  atr10/atr100  \
date       symbol                                             
2020-04-17 2382.HK       27733878.0  5.994404      1.078749   
           2388.HK       42025767.0  0.751275      1.169316   
           2628.HK      290379202.0  0.557790      1.012916   
           3328.HK       73425627.0  0.124486      1.108433   
           3988.HK      766904691.0  0.066145      1.094492   

                    atr10/atr100_scaled50  atr10/atr20  atr10/atr20_scaled50  \
date       symbol                                                              
2020-04-17 2382.HK              -0.924712     0.927090             -1.314345   
           2388.HK               0.081783     0.998771             -0.891870   
           2628.HK              -0.555399     0.928371             -0.880266   
           3328.HK              -0.248713     0.954435             -1.028724   
           3988.HK              -0.939740     0.920030             -1.820771   

                    ...  volume_pct_chg_3  volume_pct_chg_3_scaled50  \
date       symbol   ...                                                
2020-04-17 2382.HK  ...         -0.040715                  -0.297016   
           2388.HK  ...         -0.309596                  -0.783739   
           2628.HK  ...          1.531943                   1.696208   
           3328.HK  ...         -0.009715                  -0.196118   
           3988.HK  ...         -0.163673                  -0.478081   

                    volume_pct_chg_4  volume_pct_chg_4_scaled50  \
date       symbol                                                 
2020-04-17 2382.HK          0.132785                  -0.058467   
           2388.HK         -0.018954                  -0.325480   
           2628.HK          1.996060                   3.091603   
           3328.HK         -0.171444                  -0.372203   
           3988.HK          0.073001                   0.120294   

                    volume_pct_chg_5  volume_pct_chg_5_scaled50  \
date       symbol                                                 
2020-04-17 2382.HK          0.995477                   1.131469   
           2388.HK          0.199365                   0.060324   
           2628.HK          0.709304                   0.698413   
           3328.HK         -0.248594                  -0.589103   
           3988.HK         -0.115918                  -0.353722   

                    volume_scaled50  volume_slope  volume_std50  \
date       symbol                                                 
2020-04-17 2382.HK        -0.334796  2.067630e+05  5.537126e+06   
           2388.HK        -0.680581 -6.780717e+05  7.623301e+06   
           2628.HK         0.209339 -9.987013e+05  4.083472e+07   
           3328.HK        -0.743785 -1.258297e+06  1.369470e+07   
           3988.HK        -0.853307 -7.306292e+06  1.555087e+08   

                    volume_vs_50d_high  
date       symbol                       
2020-04-17 2382.HK            0.359301  
           2388.HK            0.274275  
           2628.HK            0.226225  
           3328.HK            0.256175  
           3988.HK            0.280244  

[5 rows x 150 columns]

In [4]:
#outcomes_new = load_csv(path_pc, 'outcomes_new_features_2020-04-09.csv')

In [5]:
o_labels = outcomes_new['label_v2']

In [6]:
outcomes_new.drop('label_v2', axis=1, inplace=True)

In [7]:
#o_labels.loc[o_labels==1]

In [8]:
outcomes_new.replace(0, np.nan, inplace=True)

In [9]:
outcomes_new['label_v2']=o_labels

In [10]:
outcomes_new = outcomes_new.drop(['log_return_10', 'return_10',
                                  'log_return_5', 'return_5',
                                  'log_return_4', 'return_4',
                                  'log_return_3', 'return_3',
                                  'log_return_2', 'return_2',
                                  'log_return_1', 'return_1'
                                 ], axis=1)

In [11]:
features_selected = info.features_lgb_v2

In [12]:
(len(outcomes_new.loc[outcomes_new['label_v2']==1]),len(outcomes_new.loc[outcomes_new['label_v2']==0]),len(outcomes_new.loc[outcomes_new['label_v2']==-1]))

(62259, 87982, 56012)

In [13]:
len(features_selected)

78

In [14]:
#save_csv(outcomes_new, path_pc, 'outcomes_new_features_2020-04-09.csv')

In [15]:
last_date = outcomes_new.index.get_level_values('date')[-1].to_pydatetime()
print ("Last date in dataset: ", last_date.strftime("%Y-%m-%d"))

Last date in dataset:  2020-04-17


In [16]:
test_size = 1
valid_size = 1
training_size = 2000
#valid_test_gap = 4
#         Hyperparameters with April 9th data:
#         learning_rate = 0.0010430175663616878
#         max_depth = 10
#         num_leaves = 52
#         min_data = 19
#
# max_depth = 8
# num_leaves = 77
# min_data = 18
# learning_rate = 0.0029137891706607867
#
#             Training accuracy =  0.6237882177479492
#             Validation accuracy =  0.47768670309653916
#             Test accuracy =  0.5101626016260162

In [17]:
predict_data = outcomes_new[last_date:][features_selected]

In [18]:
predict_data

delta_atr10/atr100_3_scaled50  \
date       symbol                                   
2020-04-17 0001.HK                      -0.578924   
           0002.HK                      -1.269583   
           0003.HK                      -0.863277   
           0005.HK                      -1.017026   
           0006.HK                      -0.978795   
           0011.HK                      -0.886534   
           0012.HK                      -0.563634   
           0016.HK                      -0.779981   
           0017.HK                      -0.304647   
           0019.HK                      -0.018860   
           0027.HK                       0.412607   
           0066.HK                      -0.760009   
           0083.HK                      -0.042792   
           0101.HK                      -0.520113   
           0151.HK                      -0.624895   
           0175.HK                      -0.065628   
           0267.HK                      -0.357101   
           0288.HK                      -1.124630   
           0386.HK                      -0.686864   
           0388.HK                      -0.289699   
           0669.HK                      -0.988141   
           0688.HK                      -0.580822   
           0700.HK                       0.038055   
           0762.HK                      -0.800699   
           0823.HK                      -0.425248   
           0857.HK                      -1.310908   
           0883.HK                      -0.444092   
           0939.HK                      -0.538745   
           0941.HK                      -0.738444   
           1038.HK                      -0.810824   
           1044.HK                       0.040466   
           1088.HK                      -1.486614   
           1093.HK                      -0.653791   
           1109.HK                      -0.730462   
           1113.HK                      -0.721246   
           1177.HK                      -0.211901   
           1299.HK                       0.055743   
           1398.HK                      -0.615846   
           1928.HK                       0.497586   
           1997.HK                      -0.028820   
           2007.HK                      -0.718137   
           2018.HK                      -0.847165   
           2313.HK                       0.018647   
           2318.HK                      -0.184107   
           2319.HK                      -0.438468   
           2382.HK                      -0.119683   
           2388.HK                      -0.495271   
           2628.HK                      -0.000915   
           3328.HK                      -0.403041   
           3988.HK                      -0.499790   

                    delta_atr10/atr100_10_scaled50  atr10_scaled50  \
date       symbol                                                    
2020-04-17 0001.HK                       -1.046965        0.266060   
           0002.HK                       -1.265628        0.567282   
           0003.HK                       -1.346849        0.611147   
           0005.HK                       -2.078647        0.444565   
           0006.HK                       -1.116298        0.441668   
           0011.HK                       -1.512071        0.025931   
           0012.HK                       -1.121958        0.127183   
           0016.HK                       -1.257941       -0.083286   
           0017.HK                       -1.270644       -0.150975   
           0019.HK                       -0.908392        0.192109   
           0027.HK                       -0.543464        0.390838   
           0066.HK                       -0.895760        0.441830   
           0083.HK                       -1.746706        0.744469   
           0101.HK                       -1.054317        0.625171   
           0151.HK                       -0.760058        0.592377   
           0175.HK                       -1.185579       -0.490855   
   

In [19]:
outcomes_new_dropna = outcomes_new.dropna()

In [20]:
test_date = last_date

features_selected = info.features_lgb_v2

label = 'label_v2'
    
predictions = pd.DataFrame()
        
print('Test date: ', test_date)
#print('type of test_date: ', type(test_date))

#get all trading dates from dataset of this symbol
data_dates = sorted(list(set(outcomes_new_dropna.index.get_level_values(0))))
data_converted_dates = []
for ts in data_dates:
    data_converted_dates.append(ts.to_pydatetime())

#get dates for training and valid set
start_date = data_converted_dates[-valid_size-training_size]
start_date_valid = data_converted_dates[-valid_size-1]
end_date_valid = data_converted_dates[-1]
print("start training date: ", start_date)
print("Valid dataset period: ", start_date_valid, " - ", end_date_valid)

#split into train, valid and test sets
X_y_train, X_y_valid = train_valid_split(outcomes_new_dropna, start_date, start_date_valid, end_date_valid)

#downsample the training set so each class has same number of samples
X_y_train_resampled = downsample_3class(X_y_train, label, 42)

#rename all -1 labels to 2 as LGB only takes 0,1,2...
X_y_train_resampled.loc[X_y_train_resampled[label] == -1, label] = 2
X_y_valid.loc[X_y_valid[label] == -1, label] = 2

#split into features and target sets
X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, label)
X_valid, y_valid = feature_target_split(X_y_valid, features_selected, label)

X_test = predict_data[features_selected]

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

max_acc = float("-inf")
boosting = 'gbdt'

learning_rate = 0.0029137891706607867
min_data = 18
num_leaves = 77
max_depth = 8
        
parameters = {
    'application': 'multiclass',
    'num_class': 3,
    #'is_unbalance': 'false',
    #'metric': 'auc',
    #'scale_pos_weight': 9,
    'boosting': boosting,
    'num_leaves': num_leaves,
    'feature_fraction': 0.95,
    'bagging_fraction': 0.2,
    'bagging_freq': 10,
    'learning_rate': learning_rate,
    'verbose': 0,
    'min_data_in_leaf': min_data,
    'max_depth': max_depth
}
model = lgb.train(parameters,
                       train_data,
                       valid_sets=valid_data,
                       num_boost_round=5000,
                        verbose_eval=False,
                       #feval=lgb_f1_score,
                       early_stopping_rounds=100)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_class_pred = [pred.argmax() for pred in y_pred]

X_test['pred'] = y_class_pred


Test date:  2020-04-17 00:00:00
start training date:  2011-09-23 00:00:00
Valid dataset period:  2020-04-15 00:00:00  -  2020-04-16 00:00:00


In [21]:
(len(X_y_train.loc[X_y_train['label_v2']==1]), len(X_y_train.loc[X_y_train['label_v2']==0]), len(X_y_train.loc[X_y_train['label_v2']==-1]))

(23426, 5505, 21029)

In [22]:
(len(X_y_train_resampled.loc[X_y_train_resampled['label_v2']==1]), len(X_y_train_resampled.loc[X_y_train_resampled['label_v2']==0]), len(X_y_train_resampled.loc[X_y_train_resampled['label_v2']==2]))

(5505, 5505, 5505)

In [23]:
portfolio = load_portfo_csv(path_pc, 'portfolio.csv')
short_list = []
buy_list = []
buy_df = pd.DataFrame()
short_df = pd.DataFrame()

for i, row in X_test.iterrows():
    if row['pred'] == 1: # and row['momentum_5']>0:\
#                                 and row['close_scaled50']>1.5\
#                                 and row['volume_scaled50']>0.5:
        #print('Predicted POSITIVE for {}'.format(i[1]))
        entry = {'symbol': i[1],
                'volatility': row['past_return_5_std50']}
        buy_df=buy_df.append(entry, ignore_index=True)
    elif row['pred'] == 2:# and row['momentum_5']<-60\
#                                   and row['close_scaled50']<1.5\
#                                   and row['volume_scaled50']>1\
#                                   and row['bull_ratio']<0:
        #print('Predicted NEGATIVE for {}'.format(i[1]))
        entry = {'symbol': i[1],
                'volatility': row['past_return_5_std50']}
        short_df=short_df.append(entry, ignore_index=True)

if len(buy_df)!=0 or len(short_df)!=0:
    positive_pct = len(buy_df)/(len(buy_df)+len(short_df))
    negative_pct = len(short_df)/(len(buy_df)+len(short_df))
else:
    positive_pct = 0
    negative_pct = 0

print('Postive percentage: ', positive_pct)
print('Negative percentage: ', negative_pct)
print()

if len(buy_df)>0:
    positive_list = list(buy_df['symbol'])
else:
    positive_list = []

if len(short_df)>0:
    negative_list = list(short_df['symbol'])
else:
    negative_list = []

# predictions_entry = {'date':test_date,
#                     'num_predictions_long':len(buy_df),
#                     'num_predictions_short':len(short_df),
#                     'predictions_long':positive_list,
#                     'predictions_short':negative_list,
#                     'percentage_long':positive_pct,
#                     'percentage_short':negative_pct}

# predictions=predictions.append(predictions_entry, ignore_index=True)

buy_drop_indexes = []
short_drop_indexes = []

# for i, row in buy_df.iterrows():
#     if len(portfolio)>0:
#         if symbol_converted(row['symbol']) in list(portfolio['symbol']):
#             print('####################################')
#             print('{} already in porfolio. It will not be included in buy list.'.format(row['symbol']))
#             print('####################################')
#             buy_drop_indexes.append(i)

# for i, row in short_df.iterrows():
#     if len(portfolio)>0:
#         if symbol_converted(row['symbol']) in list(portfolio['symbol']):
#             print('####################################')
#             print('{} already in porfolio. It will not be included in short list.'.format(row['symbol']))
#             print('####################################')
#             short_drop_indexes.append(i)

buy_df=buy_df.drop(buy_drop_indexes)
short_df=short_df.drop(short_drop_indexes)

#if too many predictions, shortlist them by inverse volatility

if positive_pct == 1:
    max_num_long = 6
    max_num_short = 0 
elif positive_pct >= 0.83:
    max_num_long = 5
    max_num_short = 1
elif positive_pct >= 0.67:
    max_num_long = 4
    max_num_short = 1
elif positive_pct >= 0.5:
    max_num_long = 3
    max_num_short = 2
elif positive_pct >= 0.33:
    max_num_long = 2
    max_num_short = 3
elif positive_pct >= 0.16:
    max_num_long = 1
    max_num_short = 4
else:
    max_num_long = 0
    max_num_short = 5
    
#########################################
#OVERIDING ABOVE AND SET EACH TO 5 MAX  #
#########################################
max_num_long = 5
max_num_short = 5

#Sort it so that smallest volatility comes first
if len(buy_df)>0:
    buy_df = buy_df.sort_values(by=['volatility'], ascending=True)

if len(buy_df)>max_num_long:
    buy_list = list(buy_df['symbol'][:max_num_long])
elif len(buy_df)>0:
    buy_list = list(buy_df['symbol'])
else:
    buy_list = []

#####################################
# SET BUY LIST = NULL SO IT WON"T OPEN ANY LONG POSITION
###########################
#buy_list = []

#Sort it so that smallest volatility comes first
if len(short_df)>0:
    short_df = short_df.sort_values(by=['volatility'], ascending=True)

if len(short_df)>max_num_short:
    short_list = list(short_df['symbol'][:max_num_short])
elif len(short_df)>0:
    short_list = list(short_df['symbol'])
else:
    short_list = []
print("--------------------------------")
print('Buy list: ', buy_list)
print('Short list: ', short_list)
print("--------------------------------")

Postive percentage:  0.926829268292683
Negative percentage:  0.07317073170731707

--------------------------------
Buy list:  ['2318.HK', '0011.HK', '0016.HK', '2319.HK', '0388.HK']
Short list:  ['0823.HK', '0700.HK', '1177.HK']
--------------------------------


In [24]:
short_df

,symbol,volatility
1,0823.HK,0.045203
0,0700.HK,0.054146
2,1177.HK,0.061427


In [25]:
short_list

['0823.HK', '0700.HK', '1177.HK']

In [26]:
predictions = pd.DataFrame()

trade_date_list = sorted(list(set(outcomes_new.index.get_level_values('date'))))

#max_impact_pct = 0.001
capital_base = 500000
price_list = get_price_list(outcomes_new, test_date)

if len(buy_list)>0:
    weights = inverse_volatility_weights(buy_list, test_date, outcomes_new)
    for symbol_to_buy in buy_list:
        percentage = weights[symbol_to_buy]
        traded_price = price_list[:,symbol_to_buy].values[0]
        num_shares = (percentage * capital_base)/traded_price
        print(num_shares)
        num_lots = num_shares // info.board_lots[symbol_to_buy]
        if num_lots <= 0:
            print ('Percentage of captital is not enough to pay for 1 lot of {}. '.format(symbol_to_buy))
            print ('Order cancelled')
            break
        num_shares = num_lots * info.board_lots[symbol_to_buy]
        print(num_shares)
        
        predictions=predictions.append({'date':test_date,
                          'date_close':info.trading_days[info.trading_days.index(test_date)+4],
                          'symbol':symbol_converted(symbol_to_buy),
                          'order_type':'BUY',
                          'num_shares':num_shares,
                          'target_upper':to_tick_price(outcomes_new.loc[(test_date,symbol_to_buy)]['target_upper_v2']),
                          'target_lower': to_tick_price(outcomes_new.loc[(test_date,symbol_to_buy)]['target_lower_v2'])}, ignore_index=True)

if len(short_list)>0:
    weights = inverse_volatility_weights(short_list, test_date, outcomes_new)
    for symbol_to_short in short_list:
        percentage = weights[symbol_to_short]
        traded_price = price_list[:,symbol_to_short].values[0]
        num_shares = (percentage * capital_base)/traded_price
        print(num_shares)
        num_lots = num_shares // info.board_lots[symbol_to_short]
        if num_lots <= 0:
            print ('Odd lot for SHORTING {}'.format(symbol_to_short))
            print ('Order cancelled')
            break
        num_shares = num_lots * info.board_lots[symbol_to_short]
        print(num_shares)
        
        predictions=predictions.append({'date':test_date,
                          'date_close':info.trading_days[info.trading_days.index(test_date)+4],
                          'symbol':symbol_converted(symbol_to_short),
                          'order_type':'SELL',
                          'num_shares':num_shares,
                          'target_upper':to_tick_price(outcomes_new.loc[(test_date,symbol_to_short)]['target_upper_v2']),
                          'target_lower': to_tick_price(outcomes_new.loc[(test_date,symbol_to_short)]['target_lower_v2'])}, ignore_index=True)

1343.0535615204517
1000.0
747.8174184952977
700.0
919.3843760407798
500.0
3489.4004882227305
3000.0
379.0988722118459
300.0
2762.7618437084025
2700.0
396.4216988626407
300.0
12402.706962633521
12000.0


In [27]:
portfolio

,cost,date_close,date,position,symbol,target_lower,target_upper


In [28]:
predictions

,date,date_close,num_shares,order_type,symbol,target_lower,target_upper
0,2020-04-17,2020-04-23,1000.0,BUY,2318,75.40,81.80
1,2020-04-17,2020-04-23,700.0,BUY,11,130.40,143.60
2,2020-04-17,2020-04-23,500.0,BUY,16,104.10,113.70
3,2020-04-17,2020-04-23,3000.0,BUY,2319,26.75,29.55
4,2020-04-17,2020-04-23,300.0,BUY,388,236.60,257.40
5,2020-04-17,2020-04-23,2700.0,SELL,823,67.25,73.55
6,2020-04-17,2020-04-23,300.0,SELL,700,389.20,430.00
7,2020-04-17,2020-04-23,12000.0,SELL,1177,10.84,12.24


In [29]:
weights

{'0823.HK': 0.3889968675941431,
 '0700.HK': 0.32474865570827527,
 '1177.HK': 0.28625447669758164}

In [30]:
save_csv(predictions, path_pc, 'predictions_'+test_date.strftime("%Y-%m-%d")+'.csv')